# Phase Correlation

Necessary libraries

In [ ]:
#phase correlation libraries
import numpy as np
from scipy import misc
from argparse import ArgumentParser

#complementary labraries
from google.colab import drive
import sys
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd

Mounting my repository

In [ ]:
drive.mount('/content/drive', force_remount=True)

sys.path.append('/content/drive/My Drive/repo/codes/')
from utils import video_utils

Mounted at /content/drive


Necessary functions

In [ ]:
def scale_img(src, scale):
    # calculate new dimensions
    new_w = int(src.shape[1] * scale)
    new_h = int(src.shape[0] * scale)

    return cv2.resize(src, (new_w, new_h))

In [ ]:
def phase_correlation(img, img_offset, scale=None):
    """Image translation registration by cross-correlation.
    It obtains an estimate of the cross-correlation peak by an FFT.
    It is very similar to `skimage.feature.register_translation`.
    However, our function runs faster because we restrict it to our application.
    Args:
        img (array): image.
        img_offset (array): offset image. Must have the same dim as `img`.
        scale (float, optional): If not `None`, rescale input images to run faster.
        Defaults to None.
    Returns:
        array:  shift vector (in pixels) required to register `img_offset` with
        `img`.  Axis ordering is consistent with numpy (e.g. Z, Y, X)
    """

    if img.shape != img_offset.shape:
        raise ValueError("Error: images must be same size")

    # if images in BGR, tranform to grayscal and use fft2
    # which is much faster than using fftn
    # if img.ndim > 2:
    #     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # if img_offset.ndim > 2:
    #     img_offset = cv2.cvtColor(img_offset, cv2.COLOR_BGR2GRAY)

    img = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    img_offset = np.float32(cv2.cvtColor(img_offset, cv2.COLOR_BGR2GRAY))

    if scale is not None:
        img = scale_img(img, scale)
        img_offset = scale_img(img_offset, scale)

    (x, y), c = cv2.phaseCorrelate(img, img_offset)

    # img_fft = np.fft.fft2(img)
    # img_offset_fft = np.fft.fft2(img_offset)

    # shape = img_fft.shape
    # midpoints = np.array([np.fix(axis_size / 2) for axis_size in shape])

    # R = img_fft * img_offset_fft.conj()
    # # R /= np.absolute(R)  # normalize give wrong results for large frames interval (???)
    # # (print('norm'))
    # cross_correlation = np.fft.ifft2(R)

    # shifts = np.unravel_index(np.argmax(np.absolute(cross_correlation)), shape)
    # shifts = np.array(shifts, dtype=np.float64)
    # shifts[shifts > midpoints] -= np.array(shape)[shifts > midpoints]

    if scale is not None:
        x /= scale
        y /= scale

    return (x, y), c

Generating video object

In [ ]:
video_path = '/content/drive/My Drive/repo/Excidio Mosquitoes/Dataset/2019-06-01_tubiacanga'
video_name = 'DJI_0006.MOV'
video = os.path.join(video_path, video_name)
totalFrames = video_utils.videoObj(videopath=video).videoInfo.getNumberOfFrames()

In [ ]:
counter = 0
r_array = []
x = []
y = []
c = []
video_capture = cv2.VideoCapture(video)
aux = video_capture.read()
while counter < totalFrames - 2:
  frameA = aux
  frameB = video_capture.read()
  aux = frameB
  results = phase_correlation(frameA[1], frameB[1], scale = 0.4)
  r_array.append(results)
  x.append(results[0][0])
  y.append(results[0][1])
  c.append(results[1])
  counter += 1
print(len(r_array))

7856


# Datagram

In [ ]:
x_dataframe = pd.DataFrame(x)
x_dataframe.columns = ['x']
y_dataframe = pd.DataFrame(y)
y_dataframe.columns = ['y']
c_dataframe = pd.DataFrame(c)
c_dataframe.columns = ['c']
r_dataframe = pd.concat([x_dataframe, y_dataframe,c_dataframe], axis=1)

In [ ]:
r_dataframe

,x,y,c
0,0.407954,27.894525,0.642195
1,0.345023,27.727847,0.650097
2,0.215085,27.822124,0.647268
3,0.552312,27.916742,0.655054
4,0.426195,28.050108,0.661009
...,...,...,...
7851,0.121545,23.795570,0.811877
7852,0.101397,23.531686,0.858672
7853,-0.137545,23.103048,0.888666
7854,0.036892,22.745888,0.912389


In [ ]:
r_dataframe.to_csv('phase_correlation.csv', index=False)

In [ ]:
! cat phase_correlation.csv

In [ ]:
! cp phase_correlation.csv '/content/drive/My Drive/repo/phase_correlation.csv'

## Notes

Eu quero que as imagens consigam mais ou menos se correlacionar em 20% de borda. Sabe? Então tipo assim.
Eu quero fazer a correlação do primeiro frame com o segundo.
Se forem imagens muito parecidas o pico da correlação tem que ser próximo de zero (Imagens quase não são defasadas)

Conforme a defasagem das imagens for aumentando, o pico vai apresentar uma crescente linear. Quando o shift entrar na tolerância de borda de imagem que quero, significa que os frames corrrespondem a frames consecutivos da separação.

Tomar cuidado com o vetor de direção do drone. Nas bordas, haverá uma alteração na direção desse vetor, ao detectar essa alteração haverá muita distinção entre as imagens caso não tenha chegado na margem estabelecida entre frames consecutivos.